In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from basegame.game import Game
from basegame.player import Player

In [2]:
from benchmarks.simplePlayer import SimplePlayer
from benchmarks.simpleNoCheck import SimpleNoCheck
from benchmarks.simpleAlwaysCheck import SimpleAlwaysCheck
from benchmarks.honestPlayer import HonestPlayer

from moje_dzieci.MyFirstPlayer import MyFirstPlayer
from moje_dzieci.MySecondPlayer import MySecondPlayer
from moje_dzieci.MyThirdPlayer import MyThirdPlayer
from absolutnie_do_wywalenia.cardCounter import FirstCardCounter
from absolutnie_do_wywalenia.cardCounter_base2 import SecondCardCounter
from czyje.Kondys_Dabrowski import Kondys_Dabrowski

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 1), (11, 0), (9, 2), (12, 3), (13, 0), (14, 3), (10, 0), (10, 3)]
[(12, 1), (14, 1), (11, 3), (10, 1), (14, 0), (11, 2), (12, 2), (13, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(results[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                # print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
from moje_dzieci.Kot_4 import Kot4
from moje_dzieci.Kot_5 import Kot5
from moje_dzieci.Kot_6 import Kot6
from moje_dzieci.Kociak import Kociak
from moje_dzieci.Kociak_simple import KociakSimple
from moje_dzieci.Kociak_proba import KociakProba
from moje_dzieci.Kociak_proba2 import KociakProba2
from moje_dzieci.Kociak_proba3 import KociakProba3
from moje_dzieci.Kociak_adapt import KociakAdapt
from czyje.Rafalek import Rafalek
from czyje.Janicki_Glowczewski_v2 import Janicki_Glowczewski_v2
from moje_dzieci.theKotter import Kotter
from ostatnie_pokolenie.Kotterek import Kotterek
from ostatnie_pokolenie.Kotterek2 import Kotterek2
from ostatnie_pokolenie.Kotterek3 import Kotterek3
from ostatnie_pokolenie.Kot_Kilianczyk import Kot_Kilianczyk
from ostatnie_pokolenie.Kot_Kilianczyk_random import Kot_Kilianczyk_random

from okropni.Rzucacz import Rzucacz  # gram w oszusta na wyższym poziomie ;) (tzn jak przegrywam, to error, więc remis)
# from okropni.IJustWin import IJustWin

In [10]:
strategy = [
    # ["RandomPlayer", "RANDOM", RandomPlayer],
    # ["SimplePlayer", "SimplePlayer", SimplePlayer],
    # ["NoCheck", "NoCheck", SimpleNoCheck],
    # ["AlwaysCheck", "AlwaysCheck", SimpleAlwaysCheck],
    # ["HonestPlayer", "HonestPlayer", HonestPlayer],
    # ["Kot_First", "Kot_First", MyFirstPlayer],
    # ["Kot_Second", "Kot_Second", MySecondPlayer],
    # ["Kot_Third", "Kot_Third", MyThirdPlayer],
    ["kacper","Kacper",Kondys_Dabrowski],
    ["Kot4","Kot4", Kot4],
    # ["Kot5","Kot5", Kot5],
    # ["Kot6","Kot6", Kot6],
    # ["KociakSimple","KociakSimple", KociakSimple],
    # ["Kociak_proba","Kociak_proba", KociakProba],
    # ["Kociak_proba2","Kociak_proba2", KociakProba2],
    # ["Kociak_proba3","Kociak_proba3", KociakProba3],
    # ["Kociak_adapt","Kociak_adapt", KociakAdapt],

    # ["Rzucacz","Rzucacz", Rzucacz],
    # ["IJustWin", "IJustWin", IJustWin],


    ["Rafalek","Rafalek",Rafalek],
    ["Staszek", "Staszek", Janicki_Glowczewski_v2],
    # ["Kotter","Kotter", Kotter]\
    ["Kotterek","Kotterek", Kotterek],
    ["Kotterek2","Kotterek2", Kotterek2],
    ["Kotterek3","Kotterek3", Kotterek3],
    ["KotKilianczyk","KotKilianczyk", Kot_Kilianczyk],
    ["KotKilianczyk_random","KotKilianczyk_random", Kot_Kilianczyk_random],

   ]

#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    # print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        # print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results


def generate_ranking(full_results, strategy):
    n = len(strategy)
    total_wins = [0 for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                total_wins[i] += wins_i
                total_wins[j] += wins_j

    # Sparuj nazwy z wynikami
    ranking = list(zip([s[0] for s in strategy], total_wins))

    # Posortuj malejąco
    ranking.sort(key=lambda x: x[1], reverse=True)

    maxxx = (len(strategy) -1) * 1000

    print("RANKING GRACZY (według liczby wygranych):")
    for idx, (name, wins) in enumerate(ranking, 1):
        print(f"{idx}. {name} - {wins} / {maxxx} wygranych")

    return ranking

ranking = generate_ranking(full_results, strategy)

RANKING GRACZY (według liczby wygranych):
1. KotKilianczyk_random - 4587 / 8000 wygranych
2. Kotterek3 - 4546 / 8000 wygranych
3. KotKilianczyk - 4536 / 8000 wygranych
4. Rafalek - 4364 / 8000 wygranych
5. Kotterek - 4361 / 8000 wygranych
6. Kotterek2 - 4173 / 8000 wygranych
7. kacper - 3265 / 8000 wygranych
8. Staszek - 3169 / 8000 wygranych
9. Kot4 - 2999 / 8000 wygranych


In [11]:
import pandas as pd
import numpy as np

def create_win_matrix(full_results, strategy):
    names = [s[0] for s in strategy]
    n = len(names)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = np.nan  # Brak pojedynków z samym sobą
            elif i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                matrix[i][j] = wins_i
                matrix[j][i] = wins_j

    df = pd.DataFrame(matrix, index=names, columns=names)
    print("MACIERZ WYGRANYCH:")
    print(df.round(1))

    return df

pd.set_option('display.width', 1000)
df = create_win_matrix(full_results, strategy)

MACIERZ WYGRANYCH:
                      kacper   Kot4  Rafalek  Staszek  Kotterek  Kotterek2  Kotterek3  KotKilianczyk  KotKilianczyk_random
kacper                   NaN  793.0    363.0    269.0     345.0      401.0      342.0          372.0                 380.0
Kot4                   207.0    NaN    307.0    724.0     313.0      321.0      372.0          378.0                 377.0
Rafalek                637.0  693.0      NaN    702.0     482.0      499.0      426.0          457.0                 468.0
Staszek                731.0  276.0    298.0      NaN     335.0      355.0      427.0          385.0                 362.0
Kotterek               655.0  687.0    518.0    665.0       NaN      523.0      439.0          451.0                 423.0
Kotterek2              599.0  679.0    501.0    645.0     477.0        NaN      424.0          423.0                 425.0
Kotterek3              658.0  628.0    574.0    573.0     561.0      576.0        NaN          484.0                 492